Input: Dataset with coordinates

Output: Dataset with coordinates and feature "in_US"; True means "In Continental United States", False means "Not In Continental United States"

In [ ]:
# %pip install geopy

In [ ]:
import pandas as pd
import numpy as np
import time
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import tqdm
from tqdm.notebook import tqdm_notebook

In [ ]:
filename = "hydrated_geotagged_all_tweets.csv"  # not available for publishing because of Twitter's Terms of Service
output_path = "df_with_mask.csv"

In [ ]:
df = pd.read_csv(filename, dtype=str)

In [ ]:
# get rid of all rows with no coordinates
mask = []
for item in df.coordinates:
    if str(item) == "nan":
        mask.append(False)
    else:
        mask.append(True) 

df = df[mask]

In [ ]:
# define points of interest in United States

# http://www.cohp.org/extremes/extreme_points.html
# https://www.latlong.net/degrees-minutes-seconds-to-decimal-degrees (conversion)

west = -124.73004  # westernmost point of continental united states
east = -66.94975  # easternmost point of continental united states

# US north border
north_north = 49
north_south = 41.66470503
split = -95.14984131 

# US south border
south_north = 32.71335535
south_south = 24.51713945

In [ ]:
locator = Nominatim(user_agent="myGeocoder")

def in_US(lat, lng):  # function is given coordinates, output is True if in US, False otherwise
    if lng > east or lng < west:
        return False
    elif lat > north_north or lat < south_south:
        return False
    elif lat > south_north and lat < north_south:
        return True
    elif lng < split and lat > north_south:
        return True
    else:  # need to lookup
        try:
            location = locator.reverse(str(lat)+","+str(lng))
            if location.raw['address']['country_code'] == "us":
                return True
            else:
                return False
        except:
            return None

In [ ]:
# create new feature
df["in_US"] = mask

In [ ]:
# how many of each?
df.groupby(["in_US"]).count()

In [ ]:
# save to output
df.to_csv(output_path)